In [7]:
# from sklearn import metrics,model_selection,preprocessing,linear_model,ensemble,decomposition,tree
from sklearn.metrics import mean_squared_error,mean_absolute_error, roc_auc_score,f1_score,roc_curve
from sklearn.preprocessing import StandardScaler, RobustScaler,MinMaxScaler,PolynomialFeatures
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder,OrdinalEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split, KFold ,GridSearchCV
from sklearn.model_selection import train_test_split
#from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from catboost import CatBoostRegressor,CatBoostClassifier
from lightgbm import LGBMRegressor,LGBMClassifier
from xgboost import XGBRegressor,XGBClassifier
import geopandas as gpd
import pandas as pd
import folium
!pip install geopy
from geopy.geocoders import Nominatim


import gc,itertools, pickle, re,time,warnings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
from random import choice, choices
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
#pip install pandas-profiling
import pandas_profiling
import seaborn as sns
%matplotlib inline
import numpy as np
import os 

from functools import reduce
from tqdm import tqdm
from itertools import cycle
from matplotlib_venn import venn2

import xgboost as xgb
import lightgbm as lgb
from scipy import stats
from functools import reduce
seed=777
plt.style.use("ggplot")
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]

In [10]:
import sklearn
import geopandas
import pandas as pd
import folium
import geopy
import xgboost
import lightgbm
import optuna
#import reverse_geocoder as rg
import ee

modules = [sklearn, geopandas, pd, folium, geopy, xgboost, lightgbm, optuna,  ee]

for module in modules:
    print(f"{module.__name__}: {module.__version__}")


sklearn: 1.3.1
geopandas: 0.13.2
pandas: 1.5.3
folium: 0.14.0
geopy: 2.4.0
xgboost: 1.7.5
lightgbm: 3.3.5
optuna: 3.2.0
ee: 0.1.368


In [2]:
# Read the training data from 'Train.csv' into a DataFrame called 'train'
train = pd.read_csv("Train.csv")

# Read the test data from 'Test.csv' into a DataFrame called 'test'
test = pd.read_csv("Test.csv")

# Print the shapes (number of rows and columns) of the train and test DataFrames
print(train.shape, test.shape)

# Display the first few rows of the 'train' DataFrame to inspect the data
train.head()


(1500, 4) (1500, 3)


,ID,Lat,Lon,Target
0,ID_IKZ97LLTDN8W,14.102382,33.319233,0
1,ID_K5SS5L8BA8TF,14.110377,33.333426,1
2,ID_KM39ZSIZCC5A,14.112084,33.347350,0
3,ID_JRBSKS8MYPU6,14.113252,33.314202,0
4,ID_ERVODRIN6NQJ,14.119091,33.237935,0


# DATA CLEANING

In [3]:
# Define a function 'rename_columns' that renames DataFrame columns to lowercase with underscores
def rename_columns(df):
    # Use the 'rename' method with a lambda function to convert column names
    # to lowercase and replace spaces with underscores
    df.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)
    return df

# Apply the 'rename_columns' function to the 'train' DataFrame
train = rename_columns(train)

# Apply the 'rename_columns' function to the 'test' DataFrame
test = rename_columns(test)

# FEATURE ENGINEERING

In [4]:
# Define a function 'get_country' to assign countries based on longitude ranges
def get_country(df):
    # Filter rows for Afghanistan based on longitude range and add 'country' column
    afghan = df.loc[(df['lon'] > 55) & (df['lon'] < 100)]
    afghan["country"] = "afghan"
    
    # Filter rows for Iran based on longitude range and add 'country' column
    iran = df.loc[(df['lon'] > 45) & (df['lon'] < 55)]
    iran["country"] = "iran"
    
    # Filter rows for Sudan based on longitude range and add 'country' column
    sudan = df.loc[(df['lon'] > 30) & (df['lon'] < 40)]
    sudan["country"] = "sudan"
    
    # Concatenate the filtered DataFrames vertically to combine countries
    df = pd.concat([sudan, iran, afghan], axis=0)

    return df

# Apply the 'get_country' function to the 'train' DataFrame
train = get_country(train)

# Apply the 'get_country' function to the 'test' DataFrame
test = get_country(test)

# Display the first few rows of the modified 'train' DataFrame
train.head()

,id,lat,lon,target,country
0,ID_IKZ97LLTDN8W,14.102382,33.319233,0,sudan
1,ID_K5SS5L8BA8TF,14.110377,33.333426,1,sudan
2,ID_KM39ZSIZCC5A,14.112084,33.347350,0,sudan
3,ID_JRBSKS8MYPU6,14.113252,33.314202,0,sudan
4,ID_ERVODRIN6NQJ,14.119091,33.237935,0,sudan


In [5]:
# Save filtered data for Iran
train[train["country"] == "iran"].iloc[:, 0:4].to_csv("train_iran.csv", index=False)
test[test["country"] == "iran"].iloc[:, 0:3].to_csv("test_iran.csv", index=False)

# Save filtered data for Sudan
train[train["country"] == "sudan"].iloc[:, 0:4].to_csv("train_sudan.csv", index=False)
test[test["country"] == "sudan"].iloc[:, 0:3].to_csv("test_sudan.csv", index=False)

# Save filtered data for Afghanistan
train[train["country"] == "afghan"].iloc[:, 0:4].to_csv("train_afghan.csv", index=False)
test[test["country"] == "afghan"].iloc[:, 0:3].to_csv("test_afghan.csv", index=False)